In [57]:
import time
from copy import deepcopy
from hashlib import sha256

## Hash

Create hash digest from string, int or float.

In [69]:
sha = lambda x: '0x'+sha256(str(x).encode()).hexdigest(); sha('satoshi')

'0xda2876b3eb31edb4436fa4650673fc6f01f90de2f1793c4ec332b2387b09726f'

Create random hash for testing.

In [74]:
rh = lambda: sha(time.time())
h  = rh(); h

'0x58f58b338268d9b7b9ec98e137009e853a3a0e2fc94d118dba72c62b1eb80ec1'

Turn hash into one of 256 possible emojis. 

In [75]:
def hash2emoji(h):
    if h[:2]!='0x': h='0x'+h
    offset  = int(h[0:4], 0)
    unicode = b'\U' + b'000' + str(hex(0x1F466+offset))[2:].encode()
    return unicode.decode('unicode_escape')

hash2emoji(h)

'💾'

Print hash in a nice way.

In [77]:
def ph(h):
    if len(h)<12: return h
    else        : return hash2emoji(h)+' '+h[:12] + '...' + h[-3:]

ph(h)

'💾 0x58f58b3382...ec1'

## Transaction

A transaction (abbr. tx) is used for transfering `value` from one address `fr` to another `to`.

The `fee` goes to the miner who included the tx in a block. More on mining in [03_block.ipynb](https://github.com/SharifElfouly/blockchain-from-scratch/blob/main/04_mine.ipynb). The higher it is, the more likely it is included in a block.

The `nonce` is the number of transactions sent from a given address. It's used to avoid replay attacks. For a more detailed explanation, see [here](https://kb.myetherwallet.com/en/transactions/what-is-nonce/). 

It's `hash` is a unique fingerprint. Every time something in the tx changes, it gets recalculated.

In [78]:
class TX: 
    def __init__(self, fr, to, value, fee, nonce): 
        self.fr, self.to = fr, to
        self.value       = value
        self.fee         = fee
        self.nonce       = nonce
        self.time        = time.time()
        
    def __setattr__(self, prop, val):
        super().__setattr__(prop, val)
        if prop!='sig': super().__setattr__('hash', sha(self.__dict__))
        
    def __str__ (self): 
        is_signed = '\nsigned:\ttrue' if hasattr(self, 'sig') else '\nsigned:\tfalse'
        return ('time:\t'   + time.ctime(self.time)+
                '\nfrom:\t' + ph(self.fr)+
                '\nto:\t'   + ph(self.to)+
                '\nvalue:\t'+ str(self.value)+' ether'
                '\nfee:\t'  + str(self.fee)  +' ether' 
                '\nnonce:\t'+ str(self.nonce)+
                '\nhash:\t' + ph(self.hash)+
                is_signed)
    
    def __bytes__(self):     return str(self.hash).encode()
    def __eq__(self, other): return self.hash == other.hash

Create random txs.

In [79]:
tx1 = TX(rh(), rh(), 12, 0.2,  0)
tx2 = TX(rh(), rh(), 6,  0.15, 0)

In [86]:
def txs2str(txs): return '\n'.join([str(tx)+'\n' for tx in txs])
print(txs2str([tx1,tx2]))

time:	Tue Mar 30 15:57:08 2021
from:	💰 0x4af919cf93...35e
to:	🔸 0xd2f7b8b307...494
value:	12 ether
fee:	0.2 ether
nonce:	0
hash:	🕊 0xe490a5d7ff...0d0
signed:	true

time:	Tue Mar 30 15:57:08 2021
from:	📴 0x8eb2c17c06...412
to:	🕚 0xf4da550a87...4a3
value:	6 ether
fee:	0.15 ether
nonce:	0
hash:	💬 0x4650791a83...6bb
signed:	false



#### Changing Tx
Every change in the object is reflected by its hash. Compare the tx hash below with the hash of unchanged tx above. The tx hash has changed. This is how we can make sure that nobody changes the tx without us knowing.

In [81]:
tx2_false_value       = deepcopy(tx2)
tx2_false_value.value = 120
print(tx2_false_value)

time:	Tue Mar 30 15:57:08 2021
from:	📴 0x8eb2c17c06...412
to:	🕚 0xf4da550a87...4a3
value:	120 ether
fee:	0.15 ether
nonce:	0
hash:	📊 0x641985979d...ef7
signed:	false


Transactions can be determined unequal by simply comparing the hashes as implemented in `__eq__`.

In [82]:
assert tx2 != tx2_false_value

#### Signing Tx
An account (implemented in [02_account.ipynb](https://github.com/SharifElfouly/blockchain-from-scratch/blob/main/02_account.ipynb)) has the ability to sign a tx. Here we simply mock a signature with a random hash. The signature `sig` is saved as an attribute.

In [83]:
tx1_signed = deepcopy(tx1)
tx1.sig    = rh(); print(tx1)

time:	Tue Mar 30 15:57:08 2021
from:	💰 0x4af919cf93...35e
to:	🔸 0xd2f7b8b307...494
value:	12 ether
fee:	0.2 ether
nonce:	0
hash:	🕊 0xe490a5d7ff...0d0
signed:	true


Signing the tx should not change its hash. Therefore it is equal to the unsigned tx.

In [84]:
assert tx1 == tx1_signed